# motion correction
This script will use "mcflirt" function of FSL to run estimate motion parameters.
<br>
Here we use the middle volume as reference.
<br>
The main outputs of this script are the realligned ts and the 6 estimated motion parameters (3 rotations + 3 traslations). These parmeters will be used to regress out the motion artificats from bold data.

-----------------------------------------------------------
Script written by Mehdi Behroozi
<br>
Biopsychology, 
<br>
Ruhr-University Bochum, Bochum, Germany
<br>
(2022.03)

-----------------------------------------------------------

In [ ]:
import glob
import os
import subprocess

data_path = '/mnt/d/Data/Pigeon/Sleep/analysis3'
bold_dirs = glob.glob('%s/sub*/*/bold.nii.gz'%(data_path))

for curr_bold in bold_dirs:
    sub_dir = os.path.dirname(curr_bold)
    print('Currect direcotry is: %s'%(sub_dir))
   
    #reg dir
    #reg_dir = os.path.join(sub_dir,'reg')
    #print(reg_dir)
    #res_dir = os.path.join(sub_dir,'results')# mc: motion correction; st: slice time correction; ss: spatial smoothing; tf: temporal filtering
    #print(res_dir)
    
    #print('\t 1)Creating results direcotry ...')
    #os.makedirs(res_dir)
    
    #print('\t 2) Copying bold to the result directory ...')
    #os.system('fslmaths %s %s/prefiltered_func_data -odt float'%(curr_bold, res_dir))
    
    # change directory to the res_dir 
    print('\t 1) Changing the working directory to %s'%(sub_dir))
    os.chdir(os.path.join(sub_dir,'results'))
    
    #motion correction

    print('\t 2) Motion Correction using "MCFLIRT" function of FSL ...')
    os.system("mcflirt -in ../bold -out bold_mcf -mats -plots -reffile reg/example_func -rmsrel -rmsabs -spline_final")
    os.system("mkdir -p mc ; mv -f bold_mcf.mat bold_mcf.par bold_mcf_abs.rms bold_mcf_abs_mean.rms bold_mcf_rel.rms bold_mcf_rel_mean.rms  mc")
    os.system("fsl_tsplot -i %s/bold_mcf.par -t 'MCFLIRT estimated rotations (radians)' -u 1 --start=1 --finish=3 -a x,y,z -w 640 -h 144 -o %s/rot.png"%('mc','mc'))
    os.system("fsl_tsplot -i %s/bold_mcf.par -t 'MCFLIRT estimated translations (mm)' -u 1 --start=4 --finish=6 -a x,y,z -w 640 -h 144 -o %s/trans.png"%('mc','mc'))
    os.system("fsl_tsplot -i %s/bold_mcf_abs.rms,%s/bold_mcf_rel.rms -t 'MCFLIRT estimated mean displacement (mm)' -u 1 -w 640 -h 144 -a absolute,relative -o %s/disp.png"%('mc','mc','mc')) 
  

### plot estimated motion parameters


In [ ]:
%matplotlib inline
%pylab inline

from IPython.display import display, Image
import glob
import os

data_path = '/mnt/d/Data/Pigeon/Sleep/analysis3'
mc_dirs = glob.glob('%s/sub*/*/results/mc'%(data_path))

for curr_mc in mc_dirs:
    print(curr_mc.split('/')[-4:-2])
    
    plt.show()
    display(Image(filename=os.path.join(curr_mc, 'rot.png' )))
    display(Image(filename=os.path.join(curr_mc, 'trans.png' )))